# BAT TTE Imaging Analysis Overview

## This notebook gives a brief overview of the capabilities of the BatAnalysis package to dynamically analyze BAT Time-tagged Event (TTE) data as it relates to the imaging capabilities of BAT. 

## Installation instructions for Beta testers:

Thanks to all who are willing to test the TTE portion of BatAnalysis. Here are some quick instructions for getting the code and installing it for access in python:
- `git clone -b TTE_analysis https://github.com/parsotat/BatAnalysis.git`
- `cd BatAnalysis`
- if BatAnalysis is already installed: `pip uninstall BatAnalysis`
- `pip install -e .`

Then in a jupyter notebook or an ipython session `import batanalysis as ba` should work.

Any issues that get brought up will be pushed to the github branch. To get these changes simply do `git pull` in the BatAnalysis directory and the next time you do `import batanalysis as ba` the changes will be implemented.

Here, we will analyze a BAT triggered GRB to exhibit the various capabilities that the BatAnalysis tool offers to interact with the TTE data, allowing users to download data, create sky images at any arbitrary time bin with any energy binning. 

Once again, we will look at GRB 211211A, a bright long GRB that was determined to be from a merger of compact objects. This GRB was detected by BAT and the refined analyses can be found here: https://swift.gsfc.nasa.gov/results/batgrbcat/GRB211211A/web/GRB211211A.html

Lets start off by importing the necessary python packages. 

In [ ]:
import batanalysis as ba
from swifttools.swift_too import ObsQuery 

import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from pathlib import Path
from astropy.io import fits
from astropy.time import Time, TimeDelta
import astropy.units as u
from astropy.coordinates import SkyCoord
import datetime
import os

### 1. Download the data

First, lets download the BAT TTE data for this event. We will use the basics of downloading Swift BAT data covered in the **Example_data_download** notebook in this directory. 

We will use the swifttools package to query over the hour that the GRB has occured to get the observations that have occured. We expect the GRB to appear here since we know that the GRB occured on 2021-12-11 with the trigger time of 13:09:59.634260 UTC. 

Documentation for swifttools can be found here: https://www.swift.psu.edu/too_api/

If you recently ran the **Example_TTE_rate_data_analysis** notebook in this directory then this directory will still exist and nothing will actually be downloaded.


In [ ]:
tstart=Time("2021-12-11T13:00:00")
tend=Time("2021-12-11T14:00:00")

obs_table=ObsQuery(begin=tstart, end=tend)
print(obs_table)

Lets programatically select this observation and download all the data in the bat and auxil directories to a temporary directory. 

In [ ]:
tmp_download_dir="/tmp/batdata/download_examples"

ba.datadir(tmp_download_dir, mkdir=True)

grb_obs=[i for i in obs_table if "GRB" in i.targname]
download = ba.download_swiftdata(grb_obs, quiet=False)

datadir=ba.datadir().joinpath(grb_obs[0].obsid)

print(download)
print(ba.datadir())

Lets take a look at the files that we have downloaded. 

In [ ]:
download[grb_obs[0].obsid]["data"]

As is shown above, we have: 
 - the auxil directory with information related to the spacecraft, 
 - the tdrss directory with information related to the tdrss downlinked information for the GRB when it triggered BAT, 
 - the rate directory which holds hardware rate information, 
 - the event directory with the TTE data
 - the housekeeping directory with information related to the BAT instrument itself
 - the masktag directory with the Swift Data Center (SDC) precomputed coded mask weights for the GRB
 - the products directory with SDC precomputed lightcurves at various time/energy binnings, pha files at predetermined timebins, and sky images
 - the survey directory which holds BAT survey data associated with this observation, which can be analyzed following the **Example_Survey_Data_Analysis** notebook
 
 In this notebook, we will focus on the imaging capability and better understand how we can interact with and reproduce the precomputed detector plane histograms (DPHs), the Detector Plane Images (DPIs), and sky image files from the SDC, and how we can construct new sky image files and access the relevant data. We will additionally delve into mosaicing (adding these images) and source detection. 

### 2. Loading the precomputed Detector Plane Histograms (DPHs)

The DPHs are not an integral part of imaging analyses with BAT, however they provide a useful introduction to ways that we will interact with the other BAT data products. 

DPHs are simply histograms of the TTE data binned in space (by detector) and time. The BatAnalysis class that handles this is called `BatDPH` which inherits many properties from the `Histogram` class that is available through the [histpy](https://histpy.readthedocs.io/en/latest/tutorials/quick-start.html#) python package. 

***NOTE: The DPH and Detector Plane Images include locations where there are no actual detector pixels in the detector plane.***

There are many intuitive things that we can do with this BatDPH object. To show some of these aspect, lets start by first loading in a precomputed DPH from the data that we have downloaded for GRB 211211A. To do this we simply use the `from_file` class method. 

In [ ]:
dph=ba.BatDPH.from_file(datadir.joinpath("bat/survey").joinpath("sw01088940000bsvabo27acg0859.dph.gz"))

We can easily plot the DPH to take a look at it:

In [ ]:
dph.plot()

We can see that we have lots of detectors that are off, as is shown in black, while the other detectors that are on have registered a few hundred counts over some amount of time and over some amount of energy. To find out what the timebins and energy bins are for the histogramming that was done here we can access the `.tbins` and `.ebins` attributes. 

In [ ]:
print(dph.tbins)
print(dph.ebins)

We can see that we have 3 timebins and 80 energy channels for this DPH. When we plotted the DPH, we projected the histogram onto it's spatial axis and added all the counts for each detector in each time and energy bin. If we want, we can plot a single energybin and a single timebin to see how the DPH changes.

In [ ]:
dph.plot(emin=14*u.keV, emax=16*u.keV, tmin=dph.tbins["TIME_START"][0], tmax=dph.tbins["TIME_STOP"][0])

We can see that the counts have significantly changed for the single energy range and time range that we are plotting. To better understand the layout of the BatDPH object, we can access the axes and print out the number of bins that we have:

In [ ]:
print(dph.axes.labels)
print(dph.nbins)

We can see that the 2 spatial dimensions are related to the detector X/Y coordinates, while everything else is temporal or energy binning related. 

### 3. Running the traditional batanalysis pipeline so we can verify our results

Here, we will run batgrbproducts on the directory so we have a set of precomputed Detector Plane Images and sky images that we can interact with, to show the various capabilities of the BatAnalysis Tools, and attempt to reproduce these results, to verify the results of the BatAnalysis Tools and show how we can use the package to conduct imaging analyses of BAT TTE data. 

In [ ]:
import heasoftpy as hsp
batgrbproduct_result_dir=datadir.parent.joinpath(f"{datadir.stem}-batgrbproduct-result")
hsp.batgrbproduct(indir=datadir, outdir=batgrbproduct_result_dir)

As we can see from the cell output, there are many default files that are produced. We have dealt with the lightcurves and the pha files in the ***Example_TTE_rate_data_analysis*** jupyter notebook. Here we will focus on the detector plane images and the sky images. 

### 4. Loading in a Precomputed Detector Plane Image (DPI)

A DPI is the same as a DPH for the case of TTE data, where the data has had the proper energy calibration applied to it. This is automatically taken care of in the batgrbproducts pipeline and the BatAnalysis Tools' BatEvent class that deals with this type of BAT data. As a result, eveything that we showed in the DPH section of the notebook applies here for DPIs, except that we use the BatDPI class to handle this type of data.

To show this we will load in a precomputed DPI file from when the GRB was initially detected, the pre-slew file where the name indicates that the time binning corresponds to the time before Swift started slewing to the location of the GRB. 

In [ ]:
preslew_dpi=ba.BatDPI.from_file(batgrbproduct_result_dir.joinpath("dpi").joinpath("sw01088940000b_preslew_4chan.dpi"))
print(preslew_dpi.tbins, preslew_dpi.exposure)
print(preslew_dpi.ebins)

We can plot the DPI projected over the energy bins, similar to what was done in the DPH section. We still see that there are large chunks of detectors that are turned off, but we also see the detectors that were illimunated by the GRB. This GRB had a partial coding of $\sim 22\%$ and we can see that this is the approximate portion of the detector plane that is illuminated. 

In [ ]:
preslew_dpi.plot()

Lets take a look at the BatDPI object. Identical to the DPH that we saw earlier, we've got 4 axes for "TIME", "DETX", "DETY", and "ENERGY". These allow different operations to be completed for them from [slicing](https://histpy.readthedocs.io/en/latest/tutorials/quick-start.html#Slice-and-project) over different axes to [projecting](https://histpy.readthedocs.io/en/latest/tutorials/quick-start.html#Slice-and-project) over them to get rid of the axis (for example integrating over time or over the DETX values)

In [ ]:
print(preslew_dpi.axes.labels)

If we were interested in the counts that we have from the 0,0th pixel, we can use the properties of the histogram class. Here we project the DPI onto the "DETX","DETY" & "ENERGY" axes and get rid of the "TIME" axis, since we dont care about this. Then we take a slice of the histogram correspondent to the DETX=0, DETY=0 pixel, selecting all energy bins, and finally get rid of the "DETX" & "DETY" axes with the final projection onto the "ENERGY" axis. We can see what the course photon count spectrum looks like for the pixel of interest by plotting it.

In [ ]:
preslew_dpi.project("DETX","DETY","ENERGY").slice[0,0,:].project("ENERGY").plot()

We can even rebin the DPI in energy if we would like. 

***NOTE: with DPIs that are loaded in from a preconstructed DPI file it is not possible to undo any energy or time binning. To get back to the original binning, the file will need to be reloaded***

In [ ]:
preslew_dpi.set_energybins(emin=100*u.keV, emax=350*u.keV)
print(preslew_dpi.ebins)
preslew_dpi.plot()

DPIs are a necessary data product to eventually produce sky images for BAT which we will touch on next. 

### 5. Loading in Precomputed BAT Sky Images

The Swift Data Center and the batgrbproducts pipelines both produce sky images by default. Lets look at one of these sky images to get a feel for manipulating them. The preslew DPI that we looked at in the prior section has an associated sky image (in 1 energy bin) which we will load into a BatSkyView object to understand how we can manipulate this data. 

In [ ]:
preslew_skyview=ba.BatSkyView.from_file(batgrbproduct_result_dir.joinpath("img").joinpath("sw01088940000b_preslew_1chan.img"))

Lets first plot the sky image. In order to do this, we access the flux sky image that we just created using the `.` notation. Then we can easily plot the flux sky image. If you look carefully, you can see the pixel that the GRB is located within in the top left of the plot. 

In [ ]:
preslew_skyview.sky_img.plot()

We can also take a look at the flux sky image's energy bin to see how it differs from the 4 channel DPI with a similar name. This flux sky image was constructed in a single 15-350 keV energy range. 

In [ ]:
print(preslew_skyview.sky_img.ebins)

This skyview object is more than an image, since it can contain many peices of information about BAT's view of the sky at a single timebin. One bit of information that it can hold for us is the partial coding image, which is currently not initalized in the SkyView. To do so we simply set the `pcodeimg_file` attribute to the precomputed partial coding image from batgrbproducts: 

In [ ]:
preslew_skyview.pcodeimg_file=batgrbproduct_result_dir.joinpath("img").joinpath("sw01088940000b_preslew.pcodeimg")

Now we can plot it to see what it looks like:

In [ ]:
preslew_skyview.pcode_img.plot()

Now the BatSkyView object has information related to the sky flux and the partial coding, however it can also hold images related to the background standard deviation and the SNR (which we will cover later on as these images are not constructed by default in batgrbproducts.)

Additionally, these sky images are in image coordinates which are not very intuitive. We can plot the sky flux image in a ra/dec coordinate projction or even a healpix projection (in galactic or icrs coordinate systems) using the convenience plotting method associated with the flux sky image of the partial coding image. This conveiently shows the BAT's FOV at a given point in time. 

In [ ]:
preslew_skyview.pcode_img.plot(projection="healpix", coordsys="icrs")
preslew_skyview.pcode_img.plot(projection="healpix", coordsys="galactic")
preslew_skyview.sky_img.plot(projection="ra/dec")

### 6. Creating Custom DPIs and SkyViews

In this section we will recreate the DPI and the SkyView that were precomputed before to verify that the pipeline works as intended. In the next section will also show how the BatAnalysis software expands on the imaging analysis that was previously possible with source detection and mosaicing. 

Our first step is to initalize the event data with a BatEvent object, similar to what we did in Section 4 of the Example_TTE_rate_data_analysis jupyter notebook.

In [ ]:
event=ba.BatEvent(grb_obs[0].obsid)

To create a DPI, we can simply access the `create_DPI` method. To get information/documentation on this method we can do:

In [ ]:
event.create_dpi?

We can see that we can pass in the timebin edges as well as the energybin edges. Lets get the preslew DPI information and use that to create our own DPI. Since we rebinned the preslew_dpi above, we have just entered the energy bin edges that we want.

In [ ]:
preslew_timebins=u.Quantity([preslew_dpi.tbins["TIME_START"][0], preslew_dpi.tbins["TIME_STOP"][0]])

In [ ]:
preslew_energybins=[ 15.,  25.,  50., 100., 350]*u.keV

In [ ]:
event_dpi=event.create_dpi(timebins=preslew_timebins, energybins=preslew_energybins)

We can now access our created DPI either through the event_dpi variable or through the event.dpis list as we do below. This list gets updated whenever a new DPI is created and the order of the list is correspondent to the order in which DPIs were created.

In [ ]:
print(event_dpi.tbins)
print(event_dpi.ebins)
event.dpis[0].plot()

With this DPI created, we can now create our skyview with the `create_skyview` method. Under the surface heasoftpy's batfftimage is being called.

In [ ]:
event_skyview=event.create_skyview(dpis=event_dpi)

Similar to the DPIs, we can access the created skyview through the event_skyview variable that we saved the output to or the event.skyviews list. If we print out the `tbins` and `ebins` attributes of the flux sky image for this skyview, we will see that they are the same as the DPI that was passed in. 

In [ ]:
print(event_skyview.sky_img.tbins)
print(event_skyview.sky_img.ebins)

There is additional information encoded in the skyview which is not produced by the batgrbproducts. These are the partial coding images, the background standard deviation image, and the SNR image. We can access these by the `pcode_img`, `background_stddev_img`, and `snr_img` attributes:

In [ ]:
event_skyview.pcode_img.plot()
event_skyview.bkg_stddev_img.plot()
event_skyview.snr_img.plot()

In the SNR image, we can clearly see that the GRB has a SNR>200. If we didnt know that this was a GRB, or if our eyes didnt catch the small pixel where the GRB is, we could use the `detect_sources` method to run source detection (utilizing batcelldetect on the sky image). This is much more thorough than the snr map that is constructed from our `create_skyview` method, which creates the background and SNR using theoretical Poisson noise. 

In [ ]:
detected_sources=event_skyview.detect_sources()


In [ ]:
detected_sources[:5]

Notice, how the table of the detected sources that is returned is sorted by the SNR. Also notice that there is an UNKNOWN source at the top. This is our GRB of interest. Finally, notice that the SNR of our GRB is significantly lower than the theoretical SNR that we saw plotted above.  

`detect_sources` ultimately calls batcelldetect when we are analyzing these sky images that are produced using the batfftimage heasoft script. batcelldetect uses a sliding cell annulus to looks at local count variations in the image to determine SNRs and the detection while also doing PSF fitting. Any sources that are detected are compared to those provided in a catalog, which by default we have used the `survey6b_2.cat` catalog that is included with the BatAnalysis Tools. If a SNR spike is measured with a large SNR and it isnt in the catalog then it is labeled UNKNOWN, which is what we see here. 

We can compare the detected source to the known coordinates of the GRB and see that our image analysis is pretty accurate.

In [ ]:
detected_coords=detected_sources[0]["SKYCOORD"]
orig_coords=SkyCoord(ra=event.ra, dec=event.dec)
print(detected_coords)
print(orig_coords)
print(detected_coords.separation(orig_coords).to(u.arcmin))

We have now shown how the imaging trigger portion of BAT works and are able to recreate this analysis on the ground.

Now, we will turn our attention to more advanced imaging operations such as mosaicing.

### 7. Mosaicing Images

Mosaicing images is the process of adding sky images together taking into account BAT's FOV at a given point in time and the statistical properties of the images. We try to suppress noise as much as possible. This process is identical to the mosaicing that is done for survey data, except there are certain systematic effects that do not need to be taken into account for the TTE data analysis, as the images are constructed over much shorter time intervals than those in the survey data. 

***NOTE: at this time the mosaicing can only be done on healpix projections of sky images.***

Here, we will show how we can mosaic images easily and detect sources in those images. This is especially useful in analyzing TTE data that was taken during a slew, when BAT's trigger is turned off. In this section, we will be reproducing the preslew image that we analyzed previously however, it will be done by mosaicing many images. We will show that the mosaic and non-mosaic results agree and that we find the GRB in the same location with nearly the same significance. 

First, we will construct a set of sky images in 1 second timebins from the start to end time of the "preslew" period. We do not need to first construct DPIs and then calculate the skyviews, instead we can go directly to calculating the skyviews. 

In [ ]:
preslew_times=np.arange(preslew_dpi.tbins["TIME_START"][0].value, preslew_dpi.tbins["TIME_STOP"][0].value+1)*u.s
print(preslew_times)

In [ ]:
preslew_skyviews=event.create_skyview(timebins=preslew_times, energybins=preslew_energybins)

In order to initalize all the skyviews for mosaicing, we need to specify that they all will use the helpix projection (which is the only way mosaicing currently works), with nside=512, and the coordsys set to galactic coordinates. 

In [ ]:
for i in preslew_skyviews:
    i.healpix_nside = 512
    i.projection = "healpix"
    i.healpix_coordsys = "galactic"

Now, we can do the mosaicing. This is simply done by adding the skyviews together with the `+` operator. This takes the proper addition of the inverse variance weighted fluxes into account as well as the conversion to normal flux/standard deviation images. 

In [ ]:
for i,count in zip(preslew_skyviews, range(len(preslew_skyviews))):
    if count==0:
        mosaic_skyview = i
    else:
        mosaic_skyview += i

With the mosaicing complete, we can run `detect_sources` which will search for SNR peaks in the healpix snr image where SNR > 6 by default. It will cross correlate the peaks with respect to the default BAT catalog file and list the closest source to the SNR pixel and the distance in degrees and the distance normalized by the FWHM of BAT's PSF (0.37413 deg).

We will see in the printed table that there are many repeated sky coordinate points for a SNR pixel. This is because we had 4 energy bin mosaiced images that we analyzed and there is repetition of the maximum SNR pixel. The ebins column of the table outlines which energy bin SNR image the SNR value coresponds to. 

In [ ]:
mosaic_detected_sources = mosaic_skyview.detect_sources(input_dict=dict(snrthresh=7))
mosaic_detected_sources

Nonetheless, we can see that the sky coordinates of the maximum SNR is very similar to the known coordinates of the GRB and the analysis of the preslew skyview by constructing a single time binned skyview. 

In [ ]:
mosaic_detected_coords=mosaic_detected_sources[0]["SNR_skycoord"]
print(mosaic_detected_coords.separation(orig_coords).to(u.arcmin))

We can also plot the SNR image for the mosaiced skyview and we will see a SNR peak at the edge of the BAT FOV at the coordinates that we expect. 

In [ ]:
mosaic_skyview.snr_img.plot(coordsys="icrs")